In [1]:
import os
import shutil
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import re

import numpy as np
import math

from sklearn.preprocessing import MinMaxScaler

#CifParser：cifを読み込むためのモジュール
from pymatgen.io.cif import CifParser
from pymatgen.analysis.diffraction.xrd import XRDCalculator

plt.rcParams["font.size"] = 25
plt.rcParams['font.sans-serif'] = ['Arial']

In [2]:
# ファイルからデータを読み込む関数
def read_data_from_file(file_path):
    with open(file_path, 'r') as file:
        data = file.read()
    return data

# データを解析して必要な情報を抽出する関数
def extract_phase_data(data):
    phases = []
    current_phase = {}
    lines = data.split('\n')
    
    for line in lines:
        if line.startswith("Local parameters and GOALs for phase"):
            if current_phase:
                phases.append(current_phase)
                current_phase = {}
            current_phase["Phase"] = line.split()[-1]
        elif line.startswith("SpacegroupNo"):
            current_phase["SpacegroupNo"] = int(line.split('=')[1])
        elif line.startswith("HermannMauguin"):
            current_phase["HermannMauguin"] = line.split('=')[1]
        elif line.startswith("XrayDensity"):
            current_phase["XrayDensity"] = float(line.split('=')[1]) # X-ray density in g/cm3:
        elif line.startswith("Rphase"):
            current_phase["Rphase"] = line.split('=')[1].strip('%')
    
    if current_phase:
        phases.append(current_phase)
    
    return pd.DataFrame(phases)

In [ ]:
# データが含まれるフォルダーのパスを定義
folder_path = "../0_Data/2_Compiled_Data/"
# サンプル名のリストが含まれるExcelファイルを読み込んでデータフレームにする
Sample_name_list = pd.read_excel(folder_path + "Sample_name_list.xlsx")
############################################################ リートベルトデータフォルダーのパスを定義
M_L_path = "../0_Data/1_Rietveld/Laboratory/Mafic"
P_L_path = "../0_Data/1_Rietveld/Laboratory/Pelitic/240514"

M_S_path = "../0_DATA/6_EPMA_XRD検証/0_Data/1_Rietveld/SPring-8/Mafic"
#P_S_path = "/Users/satoshi_matsuno/Desktop/研究/0_DATA/6_EPMA_XRD検証/0_Data/1_Rietveld/SPring-8/Pelitic/240520"
#P_S_path = "/Users/satoshi_matsuno/Desktop/研究/0_DATA/6_EPMA_XRD検証/0_Data/1_Rietveld/SPring-8/Pelitic/240919_Compile"
P_S_path = "../0_DATA/6_EPMA_XRD検証/0_Data/1_Rietveld/SPring-8/Pelitic/"

path_rietveld_folder = [M_L_path, P_L_path, M_S_path, P_S_path]
############################################################ リートベルトデータフォルダーのパスを定義


############################################################ Path_Df
# 指定されたディレクトリおよびそのサブディレクトリ内のすべての.xyファイルを見つけるためにglobを使用
path_glob = []
full_path_glob = []
for path in path_rietveld_folder:
    path_glob_result = glob.glob(path + "/*/*.lst")
    full_path_glob = full_path_glob+path_glob_result

    # Split
    path_glob_result_split = [path.split("/") for path in path_glob_result]
    path_glob = path_glob+path_glob_result

path_glob = [path.split("/") for path in path_glob]
Path_Df = pd.DataFrame([path[-1] for path in path_glob], columns=["lst"])
Path_Df["Path"] = full_path_glob
Path_Df.index = Path_Df["lst"].str.split(".", expand=True)[0]
############################################################ Path_Df
Path_Df.to_excel(folder_path + "Path_Rietveld_lst_df.xlsx")

# NAME LIST
name_SP_Mafic = Sample_name_list[Sample_name_list["Rock_Type"]=="Mafic Schist"]["SP-8"]
name_LAB_Mafic = Sample_name_list[Sample_name_list["Rock_Type"]=="Mafic Schist"]["LAB"]
name_SP_Pelitic = Sample_name_list[Sample_name_list["Rock_Type"]=="Pelitic Schist"]["SP-8"]
name_LAB_Pelitic = Sample_name_list[Sample_name_list["Rock_Type"]=="Pelitic Schist"]["LAB"]

name_SP = Sample_name_list["SP-8"]
name_LAB = Sample_name_list["LAB"]

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [4]:
# ループを回して各サンプルのデータを処理
all_data_LAB = []
all_data_SP8 = []
for SP_sample_name, Lab_sample_name in zip(name_SP, name_LAB):
    print(SP_sample_name, Lab_sample_name)

    SP_path = Path_Df.loc[SP_sample_name]["Path"]
    LAB_path = Path_Df.loc[Lab_sample_name]["Path"]

    #################################################################### LAB
    # ファイルのパス
    file_path = LAB_path
    # ファイルからデータを読み込み
    data = read_data_from_file(file_path)
    # データを解析してDataFrameに変換
    df_LAB = extract_phase_data(data)
    
    # DataFrameにサンプル名を追加
    df_LAB['SP_sample_name'] = SP_sample_name
    df_LAB['Lab_sample_name'] = Lab_sample_name
    
    # 結果をリストに追加
    all_data_LAB.append(df_LAB)
    #################################################################### LAB

    #################################################################### SP8
    # ファイルのパス
    file_path = SP_path
    # ファイルからデータを読み込み
    data = read_data_from_file(file_path)
    # データを解析してDataFrameに変換
    df_SP8 = extract_phase_data(data)
    
    # DataFrameにサンプル名を追加
    df_SP8['SP_sample_name'] = SP_sample_name
    df_SP8['Lab_sample_name'] = Lab_sample_name
    
    # 結果をリストに追加
    all_data_SP8.append(df_SP8)
    #################################################################### SP8


# 全ての結果を一つのDataFrameに統合
final_df_SP8 = pd.concat(all_data_SP8, ignore_index=True)
final_df_LAB = pd.concat(all_data_LAB, ignore_index=True)

NameError: name 'name_SP' is not defined